In [1]:
import glob
import pydicom
import numpy as np
from PIL import Image

folder = r"C:\Users\haipw\Desktop\人工智能与医学影像\datasets"

# filenames = glob.glob(f"{folder}/train/*.dcm")
# print(filenames[3])
# dcm = pydicom.dcmread(filenames[3])
# # read pixel data
# arr = dcm.pixel_array
# print(arr.shape)
# print(dcm.PatientSex)
# # list all the dicom tags
# for i in dcm.elements():
#     if type(i) == pydicom.DataElement:
#         print(i.description)
# t = arr.flatten()

""" 
读取数据集
"""
def get_dataset(path:str):
    X = []
    y = []
    filenames = glob.glob(f"{path}/*.dcm")
    for path in filenames:
        ds = pydicom.dcmread(path)
        try:
            arr = ds.pixel_array
            if arr.shape[0] == 2 and arr.shape[1] == 1024:
                arr2 = np.zeros((3,1024,256))
                arr2[0] = arr[0]
                arr2[1] = arr[1]
                arr2[2] = arr[0] + arr[1] # 
                arr2 = np.transpose(arr2,[1,2,0])
                img = np.array(Image.fromarray(arr2.astype(np.uint8)).resize((64,256)))
                X.append(img)
                y.append(np.array([1,0]) if ds.PatientSex.lower() == "f" else np.array([0,1]))
            else:
                print(arr.shape)
        except:
            print(path)
    X = np.stack(X)
    y = np.stack(y)
    return X,y

x_train, y_train = get_dataset(folder + "/train")
x_test, y_test = get_dataset(folder + "/test")
print(x_train.shape, x_test.shape)

(2, 256, 256)
(256, 256)
(256, 256)
C:\Users\haipw\Desktop\人工智能与医学影像\datasets/train\90243_20190820150959432000.dcm
C:\Users\haipw\Desktop\人工智能与医学影像\datasets/train\90247_20190820145151365000.dcm
(659, 256, 64, 3) (80, 256, 64, 3)


In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Input,Flatten, GlobalMaxPooling2D
from tensorflow.keras import layers, models, Sequential, backend
base_model = tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_shape=(256,64,3)
)
#  (None, 32, 8, 1024)
img_input = Input(shape=(256,64,3))
x = base_model(img_input)
x = GlobalMaxPooling2D(name='final_pooling')(x)
x = Dense(2, activation='softmax', name='fc')(x)
model=models.Model(img_input, x, name='DenseNet121')

In [12]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras

def get_model(num_classes):
  model = models.Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(256, 64, 3)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))
  return model

model = get_model(2)

In [13]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 254, 62, 32)       896       
                                                                 
 conv2d_9 (Conv2D)           (None, 252, 60, 64)       18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 126, 30, 64)      0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 126, 30, 64)       0         
                                                                 
 flatten_4 (Flatten)         (None, 241920)            0         
                                                                 
 dense_8 (Dense)             (None, 128)               30965888  
                                                      

In [17]:
batch_size = 16
epochs = 10
learning_rate = 0.05
optimizer = keras.optimizers.Adadelta(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose = 1,
            validation_data=(x_test, y_test)) 

Epoch 1/10
42/42 [==============================] - 26s 613ms/step - loss: 0.7609 - accuracy: 0.6358 - val_loss: 0.5547 - val_accuracy: 0.8250
Epoch 2/10
42/42 [==============================] - 27s 633ms/step - loss: 0.6467 - accuracy: 0.6722 - val_loss: 0.4792 - val_accuracy: 0.8625
Epoch 3/10
42/42 [==============================] - 25s 590ms/step - loss: 0.5848 - accuracy: 0.7011 - val_loss: 0.4213 - val_accuracy: 0.8375
Epoch 4/10
42/42 [==============================] - 25s 589ms/step - loss: 0.5642 - accuracy: 0.6980 - val_loss: 0.4844 - val_accuracy: 0.8375
Epoch 5/10
42/42 [==============================] - 24s 567ms/step - loss: 0.5431 - accuracy: 0.7405 - val_loss: 0.5246 - val_accuracy: 0.8250
Epoch 6/10
42/42 [==============================] - 22s 519ms/step - loss: 0.5513 - accuracy: 0.7344 - val_loss: 0.4578 - val_accuracy: 0.8250
Epoch 7/10
42/42 [==============================] - 22s 520ms/step - loss: 0.4877 - accuracy: 0.7815 - val_loss: 0.4176 - val_accuracy: 0.8250

In [18]:
loss, accuracy = model.evaluate(x_test, y_test, batch_size=128)
print("loss:", loss)
print("accuracy:", accuracy)

1/1 [==============================] - 1s 1s/step - loss: 0.4380 - accuracy: 0.8375
loss: 0.4379609525203705
accuracy: 0.8374999761581421
